TUBERIAS EN SERIE
--

In [171]:
import numpy as np
import math
import sympy as sp
from scipy import optimize

In [172]:
#constantes fisicas
g = 9.81 #gravedad
visc_c = 1.14 * 10**(-6) # coeficiente de viscosidad cinematica 15 grados

In [173]:
n = 1
p = 1 #densidad del fluido

Ht = 28.5 #altura total = z2-z1
E = 1

In [174]:
# datos a colocar (d(mm), L(m), k(mm), K)
d_tuberias = np.array([
    (600, 423, 0.3, 4.2),
    (500, 174, 0.0015, 3.4),
    (300, 373, 0.0015, 5.3),
    (250, 121, 0.0015, 7.5)
])
sp.Matrix(d_tuberias)

Matrix([
[600.0, 423.0,    0.3, 4.2],
[500.0, 174.0, 0.0015, 3.4],
[300.0, 373.0, 0.0015, 5.3],
[250.0, 121.0, 0.0015, 7.5]])

In [175]:
# datos tuberias caudal extremo final (l/s)
q_tuberias = np.array([
    60,
    74,
    60,
])/1000
sp.Matrix(q_tuberias)

Matrix([
[ 0.06],
[0.074],
[ 0.06]])

In [176]:
def h_f1():
    d = 0
    for i in range(len(d_tuberias)):
        d += d_tuberias[i,1]/((d_tuberias[i,0]/1000)**5)
    h_f = Ht * (d_tuberias[0,1]/(d_tuberias[0,0]/1000)**5)/d
    return h_f
h_f1 = h_f1()
h_f1

0.5375501984623472

In [177]:
# formula velocidad tuberia 1
D1_metros = d_tuberias[0,0]/1000
k1_metros = d_tuberias[0,2]/1000

def v_1():
    m = math.sqrt(2*g*D1_metros*h_f1)
    v_1 = ((-2 * m)/math.sqrt(d_tuberias[0,1]))*\
          math.log10((k1_metros/(3.7*D1_metros))+\
                     (2.51*visc_c*math.sqrt(d_tuberias[0,1]))/(D1_metros*m))
    return v_1


v_1 = v_1()
v_1

0.9195652130414507

In [178]:
# calculo perdidas menores tuberia 1
h_m1 = d_tuberias[0,3] * v_1**2/(2*g)
h_m1

0.18101532927375474

In [179]:
# calculo caudal tuberia 1
Q_1 = math.pi/4 * D1_metros**2 * v_1
Q_1

0.2600009386008979

In [180]:
#calculo de caudales

Q_n = np.empty(len(q_tuberias)+1)
Q = 0
Q_n[0] = Q_1

for i in range(len(q_tuberias)):
    Q += q_tuberias[i]
    Q_n[i+1] = Q_1-Q


sp.Matrix(Q_n)

Matrix([
[ 0.260000938600898],
[ 0.200000938600898],
[ 0.126000938600898],
[0.0660009386008979]])

In [181]:
v_n = np.empty(len(Q_n)+1)
v_n[0] = v_1

for i in range(len(Q_n)):
    v = 4 * Q_n[i]/(math.pi*(d_tuberias[i,0]/1000)**2)
    v_n[i] = v

v_n = np.delete(v_n,-1)
sp.Matrix(v_n)

Matrix([
[0.919565213041451],
[ 1.01859641604325],
[ 1.78254864111567],
[ 1.34456008026081]])

Numero de Reynolds correspondiente a cada tubería

In [182]:
#Numero de Reynolds usa diametros en (mm)
def N_Reynolds():
    Re_n = np.empty(len(d_tuberias))
    for i in range(len(d_tuberias)):
        Re = (d_tuberias[i,0] * v_n[i])/ (visc_c*1000)
        Re_n[i] = Re
    return Re_n

Re_n = N_Reynolds()
sp.Matrix(Re_n)



Matrix([
[483981.691074448],
[446752.814054057],
[469091.747662019],
[294859.666723862]])

Hallando f con la ecuación colebrook-white

In [183]:
# funcion para hallar los f de cada tuberia
f_nc = lambda f,n : 1.0 / np.sqrt(f) + 2.0 * np.log10(
            (d_tuberias[n, 2] / d_tuberias[n, 0]) / 3.7 + 2.51 /(Re_n[n]*np.sqrt(f)))

f_n = [optimize.brentq(f_nc, 0.0000001, 1, args=(i))for i in range(len(d_tuberias))]
f_n = np.array(f_n)
sp.Matrix(f_n)

Matrix([
[0.0176914669442501],
[0.0134713699985465],
[0.0133814800214088],
[0.0145731220564714]])

In [195]:
# función para hallar h_f de cada tuberia
def h_fn():
    h_fn = np.empty(len(d_tuberias))
    h_fn[0] = h_f1
    for i in range(len(d_tuberias)):
        h_f = f_n[i] * d_tuberias[i,1] * v_n[i]**2 / ((d_tuberias[i,0]/1000) * 2 * g)
        h_fn[i] = h_f
    return h_fn
h_fn = h_fn()
sp.Matrix(h_fn)

Matrix([
[0.537550198463254],
[0.247911282963247],
[ 2.69448334171393],
[0.649919229574547]])

In [196]:
#funcion para hallar las perdidas menores h_fm
def h_mn():
    h_mn = np.empty(len(d_tuberias))
    for i in range(len(d_tuberias)):
        h_m = d_tuberias[i,3] * v_n[i]**2 / (2 * g)
        h_mn[i] = h_m
    return h_mn
h_mn = h_mn()
sp.Matrix(h_mn)

Matrix([
[0.181015329273755],
[0.179797728839904],
[0.858340580382244],
[0.691071028069938]])

In [201]:
#suma para hallar un Ht
Hn = np.sum(h_fn + h_mn)
Hn

6.0400887192808215

In [203]:
def dif_hf():
    d = 0
    for i in range(len(d_tuberias)):
        d += d_tuberias[i,1]/((d_tuberias[i,0]/1000)**5)
    dif_hf = (Ht-Hn) * (d_tuberias[0,1]/(d_tuberias[0,0]/1000)**5)/d
    return dif_hf

dif_hf = dif_hf()
dif_hf

0.423625605838502

In [204]:
h_f1 = h_f1 + dif_hf
h_f1



0.9611758043008491